# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [25]:
import torch

In [30]:
import syft as sy

In [66]:
hook = sy.TorchHook(torch)

In [78]:
model = torch.nn.Linear(2,1)

In [79]:
model

Linear(in_features=2, out_features=1, bias=True)

In [69]:
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [70]:
bob = sy.VirtualWorker(hook, id="bob")

In [71]:
alice = sy.VirtualWorker(hook, id="alice")

In [72]:
# bob.add_workers([alice, secure_worker])
# alice.add_workers([bob, secure_worker])
# secure_worker.add_workers([bob, alice])

In [80]:
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1.], [1.]], requires_grad=True)

In [81]:
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:4].send(alice)
alice_target = target[2:4].send(alice)

In [82]:
for iterations in range(5):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    
    bob_opt = torch.optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_opt = torch.optim.SGD(params=alice_model.parameters(), lr=0.1)
    
    for i in range(10):
        bob_opt.zero_grad()
        bob_pred = bob_model(bob_data)
        bob_loss = ((bob_pred - bob_target)**2).sum()
        bob_loss.backward()

        bob_opt.step()
        bob_loss = bob_loss.get().data
        bob_loss

        alice_opt.zero_grad()
        alice_pred = alice_model(alice_data)
        alice_loss = ((alice_pred - alice_target)**2).sum()
        alice_loss.backward()

        alice_opt.step()
        alice_loss = alice_loss.get().data
        alice_loss
        
    alice_model.move(secure_worker)
    bob_model.move(secure_worker)
    model.weight.data = ((alice_model.weight.data + bob_model.weight.data)/2).get()
    model.bias.data = ((alice_model.bias.data + bob_model.bias.data)/2).get()
    print("Bob Loss" + str(bob_loss) + "Alice Loss" + str(alice_loss))

Bob Losstensor(0.0007)Alice Losstensor(0.0005)
Bob Losstensor(0.0007)Alice Losstensor(0.0002)
Bob Losstensor(0.0006)Alice Losstensor(0.0001)
Bob Losstensor(0.0004)Alice Losstensor(8.3328e-05)
Bob Losstensor(0.0003)Alice Losstensor(6.0319e-05)


In [83]:
preds = model(data)

In [90]:
loss = ((preds - target)**2).sum()

In [85]:
preds

tensor([[0.0901],
        [0.0834],
        [0.8879],
        [0.8812]], grad_fn=<AddmmBackward>)

In [86]:
target

tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)

In [91]:
loss

tensor(0.0418, grad_fn=<SumBackward0>)

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [13]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [1]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [2]:
(bob_x_share + alice_x_share) % Q

5

In [4]:
x = 5

Q = 23740629843760239486723

bob_x_share = 20 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [19]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [ ]:
# try this project here!

In [5]:
import random

In [6]:
Q = 3423523598092402

In [7]:
x = 5

In [8]:
def encrypt(x, n_shares=3):
    shares = []
    for i in range(n_shares -1):
        shares.append(random.randint(0, Q))
    final_share = Q - (sum(shares) % Q) + x
    shares.append(final_share)
    return tuple(shares)

In [11]:
my_shares = encrypt(x)

In [12]:
my_shares

(1780006035045767, 2847302195737480, 2219738965401562)

In [10]:
def decrypt(shares):
    return sum(shares) % Q

In [13]:
decrypt(my_shares)

5

In [14]:
def add(shares_1, shares_2):
    summer = []
    for i in range(len(shares_1)):
        summer.append((shares_1[i] + shares_2[i]) % Q)
    return tuple(summer)

In [15]:
shares_1 = encrypt(6)

In [16]:
shares_2 = encrypt(10)

In [18]:
added = add(shares_1, shares_2)

In [19]:
added

(586622229980411, 109861886284922, 2727039481827085)

In [20]:
decrypt(added)

16

In [22]:
-3 % -2

-1

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [139]:
x = torch.tensor([0.1, 0.2, 0.3])

In [140]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [141]:
y = x + x

In [142]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [118]:
model = torch.nn.Linear(2,1)

In [119]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
third = sy.VirtualWorker(hook, id="third")

In [120]:
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1.], [1.]], requires_grad=True)

In [121]:
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:4].send(alice)
alice_target = target[2:4].send(alice)

In [160]:
for iterations in range(5):
    model = torch.nn.Linear(2,1)
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    
    bob_opt = torch.optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_opt = torch.optim.SGD(params=alice_model.parameters(), lr=0.1)
    
    for i in range(10):
        bob_opt.zero_grad()
        bob_pred = bob_model(bob_data)
        bob_loss = ((bob_pred - bob_target)**2).sum()
        bob_loss.backward()

        bob_opt.step()
        bob_loss = bob_loss.get().data
        bob_loss

        alice_opt.zero_grad()
        alice_pred = alice_model(alice_data)
        alice_loss = ((alice_pred - alice_target)**2).sum()
        alice_loss.backward()

        alice_opt.step()
        alice_loss = alice_loss.get().data
        alice_loss
        
    
    alice_weights = alice_model.weight.data.share(alice, bob, third).move(third)
    bob_weights = bob_model.weight.data.share(alice, bob, third).move(third)
    averaged_weights = (alice_weights + bob_weights)/2
    alice_bias = alice_model.bias.data.share(alice, bob, third).move(third)
    bob_bias = bob_model.bias.data.share(alice, bob, third).move(third)
    averaged_bias = (alice_bias + bob_bias)/2
    model.weight.data = averaged_weights.get()
    model.bias.data = averaged_bias.get()
    print("model.weight:", model.weight)
    print("Bob Loss: " + str(bob_loss) + "Alice Loss: " + str(alice_loss))

model.weight: Parameter>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:78902944743 -> alice:15253533518]
	-> (Wrapper)>[PointerTensor | me:96869996141 -> bob:52674401110]
	-> (Wrapper)>tensor([[-2607636303207756638, -1321776637418090386]])
	*crypto provider: me*
Bob Loss: tensor(0.0287)Alice Loss: tensor(0.0302)
model.weight: Parameter>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:65096068996 -> alice:97396809824]
	-> (Wrapper)>[PointerTensor | me:67864706543 -> bob:96728443765]
	-> (Wrapper)>tensor([[-1643443852827094749, -2544558955255291443]])
	*crypto provider: me*
Bob Loss: tensor(0.0003)Alice Loss: tensor(0.0004)
model.weight: Parameter>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:57866618956 -> alice:79368809397]
	-> (Wrapper)>[PointerTensor | me:83225027604 -> bob:68281765471]
	-> (Wrapper)>tensor([[-2053957137748863341, -1714028918011056295]])
	*crypto provider: me*
Bob Loss: tensor(0.0248)Alice Loss: tensor(0.0

In [150]:
preds = model(data)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
preds

In [152]:
loss = ((preds - target)**2).sum()

In [153]:
target

tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)

In [ ]:
loss